In [29]:
import pandas as pd
import os
import numpy as np
import math
from skimpy import clean_columns

from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [3]:
df = clean_columns(pd.read_csv(os.path.join(os.getcwd(),'ceo-iceplant_validation_points_2020-plot-data-2022-11.csv')))
df = df[df['flagged'] == False]

6 column names have been cleaned

In [35]:
map_class = df.pl_class

ref_class = []

for i in map_class.index:
    if df.category_non_iceplant_vegetation.loc[i] == 100:
        ref_class.append(0)
    elif df.category_iceplant.loc[i] == 100:
        ref_class.append(1)
    elif df.category_low_ndvi_impervious_surface.loc[i] == 100:
        ref_class.append(2)
    elif df.category_water.loc[i] == 100:
        ref_class.append(3)
    else:
        ref_class[j]= 100

In [36]:
unique_labels(ref_class, map_class)

array([0, 1, 2, 3])

In [37]:
np.unique(ref_class, return_counts=True)

(array([0, 1, 2, 3]), array([55, 50, 19, 18]))

In [38]:
np.unique(map_class, return_counts=True)

(array([0, 1, 2, 3]), array([78, 49,  2, 13]))

In [40]:
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
# using confusion_matrix directly we get a matrix C such that
# C_{i,j} = known to be i, predicted as  j 
# The notation in the paper is 
# n_{i,j} = predicted as i, known to be j 
# so we need to take the transpose

n = confusion_matrix(ref_class, map_class, labels=range(0,4)).T
n

array([[53,  8, 13,  4],
       [ 2, 42,  3,  2],
       [ 0,  0,  2,  0],
       [ 0,  0,  1, 12]])

In [41]:
pix_counts = pd.read_csv(os.path.join(os.getcwd(), 'rasters_2020_pixel_counts.csv'))
total_pix = sum([sum(pix_counts.n_nonice_20),
                  sum(pix_counts.n_ice_20),
                  sum(pix_counts.n_ground_20),
                  sum(pix_counts.n_water_20)])

W = []      # proportion of area mapped as class i
n_idot = [] # pixels in sample that had class i in map (predicted as i, any true class j)
U_hat = []  # estimated users' accuracy (precision for each class: TP/(TP+FP))
P_hat = []  # estimated producer's accurace (sensitiviy for each class TP/(TP+FN))

for i in range(0,4):
    W.append( sum(pix_counts.iloc[:,i]) / total_pix)
    n_idot.append(sum(n[i,:]))
    U_hat.append(n[i,i] / n_idot[i])
    

In [56]:
O = sum([W[i]*n[i,i]/n_idot[i] for i in range(0,4)])
O*100

89.39047358082178

In [55]:
var_O = sum([ W[i]**2 * U_hat[i] * (1-U_hat[i])/(n_idot[i]-1) for i in range(0,4)])
# std error of estimated overall accuracy -- paper equation (5)
np.sqrt(var_O)*100

3.7179841729531313

In [50]:
var_U_hat = [np.sqrt(U_hat[i] * (1-U_hat[i])/(n_idot[i]-1))*100 for i in range(0,4)]
var_U_hat

[5.318240510717429, 5.050762722761054, 0.0, 7.69230769230769]

In [58]:
U_hat

[0.6794871794871795, 0.8571428571428571, 1.0, 0.9230769230769231]